In [1]:
!pip install datasets
!pip install transformers
!pip install accelerate
!pip install numpy, pandas
!pip install matplotlib
!pip install sentence_transformers
!pip install huggingface_hub
!pip install lm-format-enforcer

# For vLLM
!pip install vllm
!pip install ray
!pip install packaging
!pip install typing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 26.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB

In [2]:
from huggingface_hub import login
login(token="")

# Sample

In [1]:
# Succeed Code

# Please refer to entrypoints/api_server.py for
# the complete example.
import asyncio
from vllm import AsyncLLMEngine, AsyncEngineArgs, SamplingParams
import time

engine_args = AsyncEngineArgs(
    model="google/gemma-2-9b-it",
)

# initialize the engine and the example input
engine = AsyncLLMEngine.from_engine_args(engine_args)

async def add_request(prompt, request_id, start):
    print()
    print(f"request {request_id} started")
    
    # start the generation
    final_output = None
    results_generator = engine.generate(prompt, SamplingParams(temperature=0.0, max_tokens=1000), request_id)
    async for request_output in results_generator:
        final_output = request_output

    text_outputs = [prompt + output.text for output in final_output.outputs]

    print()
    print(f"request_id: {request_id}")
    print(f"calculation time: {time.time()-start}")
    print(f"text_outputs: {text_outputs}")
    
    return text_outputs
    
start = time.time()
task1 = asyncio.create_task(add_request("What is LLM?", 0, start))
task2 = asyncio.create_task(add_request("What is Japan?", 1, start))

answer1 = await task1
answer2 = await task2

print()
print()
print("---------")
print("answer1: ", answer1)
print("answer2: ", answer2)

WARNING 10-18 06:51:51 config.py:185] gemma2 has interleaved attention, which is currently not supported by vLLM. Disabling sliding window and capping the max length to the sliding window size (4096).
INFO 10-18 06:51:55 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='google/gemma-2-9b-it', speculative_config=None, tokenizer='google/gemma-2-9b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forwar

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-18 06:52:01 model_runner.py:1067] Loading model weights took 17.2179 GB
INFO 10-18 06:52:05 gpu_executor.py:122] # GPU blocks: 3861, # CPU blocks: 780
INFO 10-18 06:52:05 gpu_executor.py:126] Maximum concurrency for 4096 tokens per request: 15.08x
INFO 10-18 06:52:10 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-18 06:52:10 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-18 06:52:35 model_runner.py:1523] Graph capturing finished in 25 secs.

request 0 started
INFO 10-18 06:52:35 async_llm_engine.py:207] Added request 0.

request 1 started
INFO 10-18 06:52:35 async_llm_engine.py:207] A

In [ ]:
import asyncio
from vllm import AsyncLLMEngine, AsyncEngineArgs, SamplingParams
import time

engine_args = AsyncEngineArgs(
    model="google/gemma-2-9b-it",
)

# initialize the engine and the example input
engine = AsyncLLMEngine.from_engine_args(engine_args)
start = time.time()

async def add_request(prompt, request_id, start):
    print()
    print(f"request {request_id} started")
    
    # start the generation
    final_output = None
    results_generator = engine.generate(prompt, SamplingParams(temperature=0.0, max_tokens=1000), request_id)
    async for request_output in results_generator:
        # print(request_output) => for streaming
        final_output = request_output

    text_outputs = [prompt + output.text for output in final_output.outputs]

    print()
    print(f"request_id: {request_id}")
    print(f"calculation time: {time.time()-start}")
    #print(f"text_outputs: {text_outputs}")

    if request_id == 0:
        
        answer2 = await add_request(f"Give me some keywords of the following sentence '''{final_output.outputs[0].text}'''", 1, start)

        text_outputs += answer2
    
    return text_outputs


async def get_answer():
    #task1 = asyncio.create_task(add_request("What is LLM?", 0, start))
    #answer1 = await task1

    answer1 = await add_request("What is LLM?", 0, start)

    print()
    print()
    print("======")
    print(answer1)


await get_answer()


In [1]:
import asyncio
from vllm import AsyncLLMEngine, AsyncEngineArgs, SamplingParams
import time

engine_args = AsyncEngineArgs(
    model="google/gemma-2-9b-it",
)

# initialize the engine and the example input
engine = AsyncLLMEngine.from_engine_args(engine_args)
start = time.time()

async def add_request(prompt, request_id, start):
    print()
    print(f"request {request_id} started")
    
    # start the generation
    final_output = None
    results_generator = engine.generate(prompt, SamplingParams(temperature=0.0, max_tokens=1000), request_id)
    async for request_output in results_generator:
        # print(request_output) => for streaming
        final_output = request_output

    text_outputs = [prompt + output.text for output in final_output.outputs]

    print()
    print(f"request_id: {request_id} ended")
    print(f"calculation time: {time.time()-start}")
    
    return final_output.outputs[0].text


async def get_answer():
    whats = ["LLM", "Japan", "Kyoto"]
    
    result = await asyncio.gather(
        *[add_request(f"What is {whats[i]}", i, start) for i in range(len(whats))]
    )

    print()
    print()
    print("=== result ===")
    print(result)


await get_answer()


WARNING 10-20 05:46:26 config.py:185] gemma2 has interleaved attention, which is currently not supported by vLLM. Disabling sliding window and capping the max length to the sliding window size (4096).
INFO 10-20 05:46:30 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='google/gemma-2-9b-it', speculative_config=None, tokenizer='google/gemma-2-9b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forwar

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-20 05:46:36 model_runner.py:1067] Loading model weights took 17.2179 GB
INFO 10-20 05:46:40 gpu_executor.py:122] # GPU blocks: 3859, # CPU blocks: 780
INFO 10-20 05:46:40 gpu_executor.py:126] Maximum concurrency for 4096 tokens per request: 15.07x
INFO 10-20 05:46:44 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-20 05:46:44 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-20 05:47:00 model_runner.py:1523] Graph capturing finished in 16 secs.

request 0 started
INFO 10-20 05:47:00 async_llm_engine.py:207] Added request 0.

request 1 started
INFO 10-20 05:47:00 async_llm_engine.py:207] A

# Tests

In [7]:
from vllm import LLMEngine

# initialize engine
engine = LLMEngine(model_config = "google/gemma-2-9b-it")
# set request arguments
example_prompt = "Who is the president of the United States?"
sampling_params = SamplingParams(temperature=0.0)
request_id = 0

# add the request to the engine
engine.add_request(
    str(request_id),
    example_prompt,
    SamplingParams(temperature=0.0))
    # continue the request processing

TypeError: LLMEngine.__init__() missing 12 required positional arguments: 'cache_config', 'parallel_config', 'scheduler_config', 'device_config', 'load_config', 'lora_config', 'speculative_config', 'decoding_config', 'observability_config', 'prompt_adapter_config', 'executor_class', and 'log_stats'

In [8]:
import argparse
from typing import List, Tuple

from vllm import EngineArgs, LLMEngine, RequestOutput, SamplingParams
from vllm.utils import FlexibleArgumentParser


def create_test_prompts() -> List[Tuple[str, SamplingParams]]:
    """Create a list of test prompts with their sampling parameters."""
    return [
        ("A robot may not injure a human being",
         SamplingParams(temperature=0.0, logprobs=1, prompt_logprobs=1)),
        ("To be or not to be,",
         SamplingParams(temperature=0.8, top_k=5, presence_penalty=0.2)),
        ("What is the meaning of life?",
         SamplingParams(n=2,
                        best_of=5,
                        temperature=0.8,
                        top_p=0.95,
                        frequency_penalty=0.1)),
    ]


def process_requests(engine: LLMEngine,
                     test_prompts: List[Tuple[str, SamplingParams]]):
    """Continuously process a list of prompts and handle the outputs."""
    request_id = 0

    while test_prompts or engine.has_unfinished_requests():
        if test_prompts:
            prompt, sampling_params = test_prompts.pop(0)
            engine.add_request(str(request_id), prompt, sampling_params)
            request_id += 1

        request_outputs: List[RequestOutput] = engine.step()

        for request_output in request_outputs:
            if request_output.finished:
                print(request_output)


def initialize_engine(args: argparse.Namespace) -> LLMEngine:
    """Initialize the LLMEngine from the command line arguments."""
    engine_args = EngineArgs.from_cli_args(args)
    return LLMEngine.from_engine_args(engine_args)


def main(args: argparse.Namespace):
    """Main function that sets up and runs the prompt processing."""
    engine = initialize_engine(args)
    test_prompts = create_test_prompts()
    process_requests(engine, test_prompts)


if __name__ == '__main__':
    parser = FlexibleArgumentParser(
        description='Demo on using the LLMEngine class directly')
    parser = EngineArgs.add_cli_args(parser)
    args = parser.parse_args()
    main(args)

usage: ipykernel_launcher.py [-h] [--model MODEL] [--tokenizer TOKENIZER]
                             [--skip-tokenizer-init] [--revision REVISION]
                             [--code-revision CODE_REVISION]
                             [--tokenizer-revision TOKENIZER_REVISION]
                             [--tokenizer-mode {auto,slow,mistral}]
                             [--trust-remote-code]
                             [--download-dir DOWNLOAD_DIR]
                             [--load-format {auto,pt,safetensors,npcache,dummy,tensorizer,sharded_state,gguf,bitsandbytes,mistral}]
                             [--config-format {auto,hf,mistral}]
                             [--dtype {auto,half,float16,bfloat16,float,float32}]
                             [--kv-cache-dtype {auto,fp8,fp8_e5m2,fp8_e4m3}]
                             [--quantization-param-path QUANTIZATION_PARAM_PATH]
                             [--max-model-len MAX_MODEL_LEN]
                             [--guided-deco

SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
from vllm import EngineArgs

engine_args = EngineArgs(
    model="google/gemma-2-9b-it",
    enable_lora=True,
    max_loras=1,
    max_lora_rank=8,
    max_cpu_loras=2,
    max_num_seqs=256
)

from vllm import LLMEngine

# initialize engine
engine = LLMEngine.from_engine_args(engine_args)
#engine = LLMEngine(model_config = "google/gemma-2-9b-it")
# set request arguments
example_prompt = "Who is the president of the United States?"
sampling_params = SamplingParams(temperature=0.0)
request_id = 0

# add the request to the engine
engine.add_request(
    str(request_id),
    example_prompt,
    SamplingParams(temperature=0.0))
    # continue the request processing

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

WARNING 10-17 05:29:04 config.py:179] Gemma 2 uses sliding window attention for every odd layer, which is currently not supported by vLLM. Disabling sliding window and capping the max length to the sliding window size (4096).
INFO 10-17 05:29:09 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='google/gemma-2-9b-it', speculative_config=None, tokenizer='google/gemma-2-9b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, col

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-17 05:29:29 model_runner.py:1071] Loading model weights took 17.2179 GB
INFO 10-17 05:29:35 gpu_executor.py:122] # GPU blocks: 3851, # CPU blocks: 780
INFO 10-17 05:29:35 gpu_executor.py:126] Maximum concurrency for 4096 tokens per request: 15.04x
INFO 10-17 05:29:38 model_runner.py:1402] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-17 05:29:38 model_runner.py:1406] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-17 05:30:15 model_runner.py:1530] Graph capturing finished in 36 secs.


In [1]:
# Please see the example/ folder for more detailed examples.
from vllm import EngineArgs, LLMEngine, SamplingParams
import time

engine_args = EngineArgs(
    model="google/gemma-2-9b-it",
)
"""
    enable_lora=True,
    max_loras=1,
    max_lora_rank=8,
    max_cpu_loras=2,
    max_num_seqs=256
)
"""

# initialize engine and request arguments
engine = LLMEngine.from_engine_args(engine_args)

example_inputs = [(i, "What is LLM?", SamplingParams(temperature=0.0, max_tokens=1000)) for i in range(10)]

finished_request_ids = []
# Start the engine with an event loop
while True:
    if example_inputs:
        req_id, prompt, sampling_params = example_inputs.pop(0)
        engine.add_request(str(req_id),prompt,sampling_params)

    # continue the request processing
    request_outputs = engine.step()
    for request_output in request_outputs:
        if request_output.finished and request_output.request_id not in finished_request_ids:
            # return or show the request output
            finished_request_ids.append(request_output.request_id)
            print(f"request {request_output.request_id} has finished")
            #print(request_output.outputs[0].text)

    time.sleep(10)

    if not (engine.has_unfinished_requests() or example_inputs):
        break

/usr/local/lib/python3.10/dist-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


WARNING 10-17 06:21:09 config.py:179] Gemma 2 uses sliding window attention for every odd layer, which is currently not supported by vLLM. Disabling sliding window and capping the max length to the sliding window size (4096).
INFO 10-17 06:21:14 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='google/gemma-2-9b-it', speculative_config=None, tokenizer='google/gemma-2-9b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, col

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-17 06:21:19 model_runner.py:1071] Loading model weights took 17.2179 GB
INFO 10-17 06:21:24 gpu_executor.py:122] # GPU blocks: 3861, # CPU blocks: 780
INFO 10-17 06:21:24 gpu_executor.py:126] Maximum concurrency for 4096 tokens per request: 15.08x
INFO 10-17 06:21:28 model_runner.py:1402] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-17 06:21:28 model_runner.py:1406] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-17 06:21:57 model_runner.py:1530] Graph capturing finished in 28 secs.
INFO 10-17 06:22:02 metrics.py:345] Avg prompt throughput: 11.9 tokens/s, Avg generation throughput: 228.4 tokens/s, Running: 10 reqs, Swapped: 0 

In [2]:
print("\n\nLLM stands for **Large Language Model**. \n\nIt's a type of artificial intelligence (AI) that excels at understanding and generating human-like text. \n\nHere's a breakdown:\n\n* **Large:** LLMs are trained on massive datasets of text and code, often containing billions or even trillions of words. This vast amount of data allows them to learn complex patterns and relationships within language.\n* **Language:** LLMs are specifically designed to work with language. They can read, understand, interpret, and generate text in various formats, including articles, stories, poems, code, and more.\n* **Model:** An LLM is a mathematical model that represents knowledge about language. It uses this knowledge to perform tasks such as:\n\n    * **Text Generation:** Writing creative content, summarizing text, translating languages, and more.\n    * **Text Comprehension:** Answering questions, identifying sentiment, and extracting information from text.\n    * **Code Generation:** Writing and debugging code in different programming languages.\n    * **Dialogue Systems:** Engaging in natural-sounding conversations with humans.\n\n**Examples of LLMs:**\n\n* GPT-3 (Generative Pre-trained Transformer 3) by OpenAI\n* LaMDA (Language Model for Dialogue Applications) by Google\n* BERT (Bidirectional Encoder Representations from Transformers) by Google\n\n**Key Features of LLMs:**\n\n* **Contextual Understanding:** LLMs can understand the context of a conversation or text passage, allowing them to generate more relevant and coherent responses.\n* **Generative Capabilities:** They can create new text that is grammatically correct and stylistically appropriate.\n* **Adaptability:** LLMs can be fine-tuned for specific tasks or domains, improving their performance in those areas.\n\n**Applications of LLMs:**\n\nLLMs have a wide range of applications, including:\n\n* **Chatbots and Virtual Assistants:** Providing automated customer service and support.\n* **Content Creation:** Generating articles, blog posts, marketing copy, and other types of content.\n* **Education:** Assisting with homework, providing personalized learning experiences, and grading essays.\n* **Research:** Analyzing large amounts of text data to uncover insights and trends.\n* **Healthcare:** Summarizing patient records, assisting with diagnosis, and providing medical information.\n\n\n\nLet me know if you have any other questions about LLMs!\n")



LLM stands for **Large Language Model**. 

It's a type of artificial intelligence (AI) that excels at understanding and generating human-like text. 

Here's a breakdown:

* **Large:** LLMs are trained on massive datasets of text and code, often containing billions or even trillions of words. This vast amount of data allows them to learn complex patterns and relationships within language.
* **Language:** LLMs are specifically designed to work with language. They can read, understand, interpret, and generate text in various formats, including articles, stories, poems, code, and more.
* **Model:** An LLM is a mathematical model that represents knowledge about language. It uses this knowledge to perform tasks such as:

    * **Text Generation:** Writing creative content, summarizing text, translating languages, and more.
    * **Text Comprehension:** Answering questions, identifying sentiment, and extracting information from text.
    * **Code Generation:** Writing and debugging code in 

In [7]:
request_output.request_id

'0'

In [3]:
!pip install fastapi


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
# Please refer to entrypoints/api_server.py for
# the complete example.
from vllm import EngineArgs, LLMEngine, SamplingParams, PoolingParams
import time

engine_args = EngineArgs(
    model="google/gemma-2-9b-it",
)

# initialize the engine and the example input
engine = AsyncLLMEngine.from_engine_args(engine_args)
example_input = {
    "input": "What is LLM?",
    "request_id": 0,
}

# start the generation
results_generator = engine.encode(
   example_input["input"],
   PoolingParams(),
   example_input["request_id"])

# get the results
final_output = None
async for request_output in results_generator:
    if await request.is_disconnected():
        # Abort the request if the client disconnects.
        await engine.abort(request_id)
        # Return or raise an error
        raise Exception("error")
    final_output = request_output
    print(final_output)

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

WARNING 10-18 00:17:33 config.py:185] gemma2 has interleaved attention, which is currently not supported by vLLM. Disabling sliding window and capping the max length to the sliding window size (4096).


AttributeError: 'EngineArgs' object has no attribute 'disable_log_requests'

In [1]:
# Please refer to entrypoints/api_server.py for
# the complete example.

from vllm import AsyncLLMEngine, EngineArgs, AsyncEngineArgs, LLMEngine, SamplingParams, PoolingParams
import time

engine_args = AsyncEngineArgs(
    model="google/gemma-2-9b-it",
)

# initialize the engine and the example input
engine = AsyncLLMEngine.from_engine_args(engine_args)
example_input = {
    "prompt": "What is LLM?",
    "stream": False, # assume the non-streaming case
    "temperature": 0.0,
    "request_id": 0,
}

# start the generation
results_generator = engine.generate(
   example_input["prompt"],
   SamplingParams(temperature=example_input["temperature"]),
   example_input["request_id"])

start = time.time()

# get the results
final_output = None
async for request_output in results_generator:  # the mechanism under this (AsyncGenerator from the typing module): https://chatgpt.com/share/6711c243-dd54-8006-9a54-3c2cb1b59ce9
    """
    if await request_output.is_disconnected():
        # Abort the request if the client disconnects.
        await engine.abort(request_id)
        # Return or raise an error
        raise Exception("error")
    """
    final_output = request_output

wrap1 = time.time()

print("1")
print(wrap1 - start)
print(final_output)

results_generator2 = engine.generate(
   example_input["prompt"],
   SamplingParams(temperature=example_input["temperature"]),
   example_input["request_id"])

async for request_output in results_generator2:  # the mechanism under this (AsyncGenerator from the typing module): https://chatgpt.com/share/6711c243-dd54-8006-9a54-3c2cb1b59ce9
    final_output = request_output

wrap2 = time.time()

print("2")
print(wrap2 - start)
# Process and return the final output
print(final_output)

WARNING 10-18 04:41:09 config.py:185] gemma2 has interleaved attention, which is currently not supported by vLLM. Disabling sliding window and capping the max length to the sliding window size (4096).
INFO 10-18 04:41:14 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='google/gemma-2-9b-it', speculative_config=None, tokenizer='google/gemma-2-9b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forwar

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-18 04:41:19 model_runner.py:1067] Loading model weights took 17.2179 GB
INFO 10-18 04:41:24 gpu_executor.py:122] # GPU blocks: 3861, # CPU blocks: 780
INFO 10-18 04:41:24 gpu_executor.py:126] Maximum concurrency for 4096 tokens per request: 15.08x
INFO 10-18 04:41:28 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-18 04:41:28 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-18 04:41:49 model_runner.py:1523] Graph capturing finished in 21 secs.
INFO 10-18 04:41:49 async_llm_engine.py:207] Added request 0.
INFO 10-18 04:41:50 async_llm_engine.py:175] Finished request 0.
1
0.781533718109130

In [1]:
# Please refer to entrypoints/api_server.py for
# the complete example.
import asyncio
from vllm import AsyncLLMEngine, AsyncEngineArgs, SamplingParams
import time

engine_args = AsyncEngineArgs(
    model="google/gemma-2-9b-it",
)

# initialize the engine and the example input
engine = AsyncLLMEngine.from_engine_args(engine_args)
example_input = {
    "prompt": "What is LLM?",
    "stream": False, # assume the non-streaming case
    "temperature": 0.0,
    "request_id": 0,
}

async def add_request(prompt, request_id, start):
    print()
    print(f"request {request_id} started")
    
    # start the generation
    final_output = None
    results_generator = engine.generate(prompt, SamplingParams(temperature=0.0, max_tokens=1000), request_id)
    async for request_output in results_generator:
        final_output = request_output

    text_outputs = [prompt + output.text for output in final_output.outputs]

    print()
    print(f"request_id: {request_id}")
    print(f"calculation time: {time.time()-start}")
    print(f"text_outputs: {text_outputs}")
    
    return text_outputs
    
start = time.time()
answer1 = asyncio.create_task(add_request("What is LLM?", 0, start))
answer2 = asyncio.create_task(add_request("What is Japan?", 1, start))

print(await answer1)
print(await answer2)

#answer1 = asyncio.run(add_request("What is LLM?", 0, start))
#answer2 = asyncio.run(add_request("What is Japan?", 1, start))

#print("answer1: ", answer1)
#print("answer2: ", answer2)

WARNING 10-18 06:43:57 config.py:185] gemma2 has interleaved attention, which is currently not supported by vLLM. Disabling sliding window and capping the max length to the sliding window size (4096).
INFO 10-18 06:44:01 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='google/gemma-2-9b-it', speculative_config=None, tokenizer='google/gemma-2-9b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forwar

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-18 06:44:06 model_runner.py:1067] Loading model weights took 17.2179 GB
INFO 10-18 06:44:11 gpu_executor.py:122] # GPU blocks: 3861, # CPU blocks: 780
INFO 10-18 06:44:11 gpu_executor.py:126] Maximum concurrency for 4096 tokens per request: 15.08x
INFO 10-18 06:44:16 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-18 06:44:16 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-18 06:44:39 model_runner.py:1523] Graph capturing finished in 23 secs.

request 0 started
INFO 10-18 06:44:39 async_llm_engine.py:207] Added request 0.

request 1 started
INFO 10-18 06:44:39 async_llm_engine.py:207] A

In [2]:
import asyncio, time

async def say_after(delay, what):
    print(f'before_sleep: {what}')
    await asyncio.sleep(delay)
    print(what)

async def execute():
    print(f'started at {time.strftime("%X")}')
    #await say_after(1, 'hello')
    say_after(1, 'hello')
    #await say_after(2, 'world')
    say_after(2, 'world')
    print(f'ended at {time.strftime("%X")}')

event_loop = asyncio.get_running_loop()
if event_loop.is_running():
    task = asyncio.create_task(execute())


started at 05:57:48
ended at 05:57:48


/tmp/ipykernel_8971/4171694787.py:11: RuntimeWarning: coroutine 'say_after' was never awaited
  say_after(1, 'hello')
/tmp/ipykernel_8971/4171694787.py:13: RuntimeWarning: coroutine 'say_after' was never awaited
  say_after(2, 'world')


In [3]:
import asyncio,time

async def my_coroutine():
    print("Hello, asyncio!")

event_loop = asyncio.get_running_loop()
if event_loop.is_running():
    task = asyncio.create_task(my_coroutine())

Hello, asyncio!


In [ ]:
import asyncio
from vllm import AsyncLLMEngine, AsyncEngineArgs, SamplingParams
import time

engine_args = AsyncEngineArgs(
    model="google/gemma-2-9b-it",
)

# initialize the engine and the example input
engine = AsyncLLMEngine.from_engine_args(engine_args)
start = time.time()

async def add_request(prompt, request_id, start):
    print()
    print(f"request {request_id} started")
    
    # start the generation
    final_output = None
    results_generator = engine.generate(prompt, SamplingParams(temperature=0.0, max_tokens=1000), request_id)
    async for request_output in results_generator:
        # print(request_output) => for streaming
        final_output = request_output

    text_outputs = [prompt + output.text for output in final_output.outputs]

    print()
    print(f"request_id: {request_id}")
    print(f"calculation time: {time.time()-start}")
    #print(f"text_outputs: {text_outputs}")

    if request_id == 0:
        #task2 = asyncio.create_task(add_request(f"Give me some keywords of the following sentence '''{final_output.outputs[0].text}'''", 1, start))
        #answer2 = await task2

        answer2 = await add_request(f"Give me some keywords of the following sentence '''{final_output.outputs[0].text}'''", 1, start)

        text_outputs += answer2
    
    return text_outputs


async def get_answer():
    #task1 = asyncio.create_task(add_request("What is LLM?", 0, start))
    #answer1 = await task1

    answer1 = await add_request("What is LLM?", 0, start)

    print()
    print()
    print("======")
    print(answer1)


await get_answer()


In [3]:
from typing import AsyncGenerator

# Define an asynchronous generator function
async def countdown(start: int) -> AsyncGenerator[int, None]:
    for i in range(start, 0, -1):
        yield i
        await asyncio.sleep(1)  # Simulate async work

# Using the asynchronous generator
import asyncio

async def main():
    async for number in countdown(5):
        print(f"Countdown: {number}")

# Run the async main function
await main()


Countdown: 5
Countdown: 4
Countdown: 3
Countdown: 2
Countdown: 1


In [2]:
import asyncio


async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f

async def execute():
    task_a = asyncio.create_task(factorial('A', 2))
    task_b = asyncio.create_task(factorial('B', 3))
    task_c = asyncio.create_task(factorial('C', 4))

    L = []
    L.append(await task_a)
    L.append(await task_b)
    L.append(await task_c)

    print(L)

async def execute2():
    # Schedule three calls *concurrently*:
    L = await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    print(L)


#asyncio.run(execute())
await execute2()


Task A: Compute factorial(2), currently i=2...
Task B: Compute factorial(3), currently i=2...
Task C: Compute factorial(4), currently i=2...
Task A: factorial(2) = 2
Task B: Compute factorial(3), currently i=3...
Task C: Compute factorial(4), currently i=3...
Task B: factorial(3) = 6
Task C: Compute factorial(4), currently i=4...
Task C: factorial(4) = 24
[2, 6, 24]


In [1]:
import asyncio, inspect


async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")

    # get the instance which called this job_instance
    current_frame = inspect.currentframe()
    caller_frame = current_frame.f_back
    caller_self = cacaller_selfller_frame.f_locals.get('self', None) # Get the instance that called the function
    print(caller_self)
    
    return f

async def execute():
    task_a = asyncio.create_task(factorial('A', 2))
    task_b = asyncio.create_task(factorial('B', 3))
    task_c = asyncio.create_task(factorial('C', 4))

    L = []
    L.append(await task_a)
    L.append(await task_b)
    L.append(await task_c)

    print(L)

async def execute2():
    # Schedule three calls *concurrently*:
    L = await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    print(L)


#asyncio.run(execute())
await execute2()


Task A: Compute factorial(2), currently i=2...
Task B: Compute factorial(3), currently i=2...
Task C: Compute factorial(4), currently i=2...
Task A: factorial(2) = 2
Task B: Compute factorial(3), currently i=3...
Task C: Compute factorial(4), currently i=3...


NameError: name 'cacaller_selfller_frame' is not defined

Task B: factorial(3) = 6
Task C: Compute factorial(4), currently i=4...
Task C: factorial(4) = 24


In [2]:
def log_method_calls(cls):
    original_methods = cls.__dict__.copy()
    for method_name, method in original_methods.items():
        if callable(method) and not method_name.startswith("__"):
            setattr(cls, method_name, log_call(method))
    return cls

def log_call(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args: {args} and kwargs: {kwargs}")
        return func(*args, **kwargs)
    return wrapper

@log_method_calls
class SuperClass:
    def method_in_superclass(self):
        print("In SuperClass method")

class SubClass(SuperClass):
    def method_in_subclass(self):
        self.method_in_superclass()

sub = SubClass()
sub.method_in_subclass()


Calling method_in_superclass with args: (<__main__.SubClass object at 0x7a4b54267a00>,) and kwargs: {}
In SuperClass method


In [6]:
from AIMO2.MakeStrategy import MakeStrategy
MakeStrategy.__name__, MakeStrategy.__module__

('MakeStrategy', 'AIMO2.MakeStrategy')